TASK VI: Quantum Representation

In [ ]:
pip install pennylane -q

Getting data of Mnist for task:

In [ ]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Settings ---
transform = transforms.Compose([
    transforms.Resize((10, 10)), # 28x28 -> 4x4
    transforms.ToTensor(),     # Convert to numbers (0 to 1)
])

# --- 2. Download Data ---
print("Downloading MNIST Data...")
# Hum sirf Training data utha rahe hain abhi
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# --- 3. Filter Data (Sirf 0 aur 1 rakhte hain shuru mein) ---
# Poore 10 digits par train karna mushkil hoga.
# Hum sirf '0' aur '1' ko compare karenge (Simple start).
idx = (dataset.targets == 0) | (dataset.targets == 1)
dataset.targets = dataset.targets[idx]
dataset.data = dataset.data[idx]

print(f"Total Images (0 and 1 only): {len(dataset.targets)}")

# --- 4. Visualize (Dekhte hain data kaisa dikh raha hai) ---
# Ek example uthate hain
image, label = dataset[0]

# Plotting
plt.figure(figsize=(4, 4))
plt.imshow(image.squeeze(), cmap='gray') # .squeeze() faltu dimensions hatata hai
plt.title(f"Label: {label} | Size: 4x4 Pixels")
plt.axis('off')
plt.show()

print("Original Pixel Values (Flattened):")
print(image.flatten().shape)

Now Using SWAP test but first apply simense network:

In [ ]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=9)

def image_embedding(pixels, weights, wires):
    pixel_batches = pixels.reshape(4, 4)

    for i in range(4): 

        for j in range(4):
            qml.RX(pixel_batches[i, j], wires=wires[j])

        # Trainable Weights (RY)
        for j in range(4):
            qml.RY(weights[i, j], wires=wires[j])

        # Entanglement (CNOT Ring)
        for j in range(4):
            qml.CNOT(wires=[wires[j], wires[(j+1) % 4]])

# --- 3. The SWAP Test Circuit (Main Code) ---
@qml.qnode(dev)
def swap_test_circuit(params, img1, img2):

    qml.Hadamard(wires=0)


    image_embedding(img1, params, wires=[1, 2, 3, 4])


    image_embedding(img2, params, wires=[5, 6, 7, 8])

    # Step C: CSWAP (Comparison)

    qml.CSWAP(wires=[0, 1, 5])
    qml.CSWAP(wires=[0, 2, 6])
    qml.CSWAP(wires=[0, 3, 7])
    qml.CSWAP(wires=[0, 4, 8])


    qml.Hadamard(wires=0)

    return qml.probs(wires=0)

print("Quantum Judge (Swap Test) is Ready! ⚖️")

Quantum Judge (Swap Test) is Ready! ⚖️


Now lets check it:

In [ ]:
# --- Test Data ---

img1 = np.random.rand(16)
img2 = np.random.rand(16) 
weights = np.random.rand(4, 4)

# --- TEST 1: Same vs Same ---
result_same = swap_test_circuit(weights, img1, img1)
print(f"Test 1 (Same Image): Similarity = {result_same[0]:.4f}")

# --- TEST 2: Same vs Different ---
result_diff = swap_qnode(weights, img1, img2)
print(f"Test 2 (Diff Image): Similarity = {result_diff[0]:.4f}")

Test 1 (Same Image): Similarity = 1.0000
Test 2 (Diff Image): Similarity = 0.8562


Traing our swap circuit with pytorch

In [ ]:
import torch

# --- 1. Setup PyTorch Interface ---
# Hum purane circuit ko PyTorch ke liye ready kar rahe hain
# diff_method="backprop" ka matlab hum gradients calculate kar sakte hain
swap_qnode = qml.QNode(swap_test_circuit, dev, interface="torch", diff_method="backprop")

# --- 2. Data Preparation Function ---
# Hum randomly 'pairs' banayenge training ke liye
def get_batch(batch_size=4):
    img1_batch = []
    img2_batch = []
    labels = []

    for _ in range(batch_size):
        # Do random images uthao
        # Use len(dataset) to ensure we stay within bounds
        idx1 = np.random.randint(0, len(dataset))
        idx2 = np.random.randint(0, len(dataset))

        # Image data extract karo using dataset[idx] to apply the 4x4 transform
        x1_img, y1 = dataset[idx1]
        x2_img, y2 = dataset[idx2]

        x1 = x1_img.flatten().float() * np.pi # Pi se multiply taaki rotation achi ho
        x2 = x2_img.flatten().float() * np.pi

        # Label check karo: Kya dono same number hain? (e.g. 0 aur 0)
        # Agar same hain -> Target = 1.0
        # Agar alag hain -> Target = 0.0
        target = 1.0 if y1 == y2 else 0.0

        img1_batch.append(x1)
        img2_batch.append(x2)
        labels.append(target)

    return torch.stack(img1_batch), torch.stack(img2_batch), torch.tensor(labels)

# --- 3. Initialize Weights (Trainable Parameters) ---
# 4 Layers, 4 Qubits
weights = torch.rand((4, 4), requires_grad=True)

# Optimizer (Adam - Sabse reliable)
opt = torch.optim.Adam([weights], lr=0.1)

# --- 4. Training Loop ---
print("Training Started... (Yeh thoda slow chalega)")

history = []
for epoch in range(20): # 20 Rounds (Demo ke liye)

    # A. Data Lao
    imgs1, imgs2, targets = get_batch(batch_size=4)

    opt.zero_grad()
    loss = 0

    # B. Har pair ke liye Circuit Run karo
    for i in range(len(imgs1)):
        # Prediction: Circuit batayega kitna same hai (Probability of |0>)
        prediction = swap_qnode(weights, imgs1[i], imgs2[i])[0]

        # Loss: (Prediction - Target)^2
        # Agar Target 0 tha aur Prediction 0.8 aayi -> Loss bada hoga
        loss += (prediction - targets[i]) ** 2

    # Average Loss
    loss = loss / len(imgs1)

    # C. Backpropagation (Seekho aur Sudharo)
    loss.backward()
    opt.step()

    history.append(loss.item())
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")

print("Training Complete! ✅")

Training Started... (Yeh thoda slow chalega)
Epoch 1: Loss = 0.0880
Epoch 2: Loss = 0.0275
Epoch 3: Loss = 0.1705
Epoch 4: Loss = 0.1439
Epoch 5: Loss = 0.1689
Epoch 6: Loss = 0.1205
Epoch 7: Loss = 0.1729
Epoch 8: Loss = 0.2106
Epoch 9: Loss = 0.1206
Epoch 10: Loss = 0.2344
Epoch 11: Loss = 0.3477
Epoch 12: Loss = 0.2473
Epoch 13: Loss = 0.1507
Epoch 14: Loss = 0.1703
Epoch 15: Loss = 0.0447
Epoch 16: Loss = 0.1244
Epoch 17: Loss = 0.4150
Epoch 18: Loss = 0.2854
Epoch 19: Loss = 0.0957
Epoch 20: Loss = 0.3535
Training Complete! ✅
